Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_psi_t_var_formula">_psi_t_var_formula</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, c, k, m, n, t, P, defaults, Function
from proveit.logic import Equals, InSet
from proveit.numbers import zero, one, two, i, e, pi, Add, Exp, Less, LessEq, Mult, Neg, subtract
from proveit.numbers import Complex, Interval, Natural
from proveit.numbers.exponentiation import (
        add_one_right_in_exp, exp_eq_for_eq_base_and_exp, exponential_monotonocity)
from proveit.numbers.number_sets.natural_numbers import fold_forall_natural_pos
from proveit.physics.quantum.QPE import _phase, _phase_is_real

In [ ]:
%proving _psi_t_var_formula

In [ ]:
defaults.assumptions = _psi_t_var_formula.all_conditions()

In [ ]:
# the induction theorem for positive naturals
fold_forall_natural_pos

In [ ]:
# instantiate the induction theorem
induction_inst = fold_forall_natural_pos.instantiate(
    {Function(P,t):_psi_t_var_formula.instance_expr, m:t, n:t})

### Some Related Properties and Definitions Needed for Later Processing
Mainly: some domains and orderings. Notice that throughout this section devoted to the induction proof, $t$ is a _variable_, not a literal.

In [ ]:
# used when processing products involving the phase phi
_phase_is_real

In [ ]:
two_pow_t_var_less_one = subtract(Exp(two, t), one)

In [ ]:
# two_pow_t_less_one_is_nat_pos

In [ ]:
two_pow_t_var_less_one.deduce_in_number_set(Natural)

In [ ]:
LessEq(zero, two_pow_t_var_less_one).prove()

In [ ]:
Less(t, Add(t, one)).prove()

In [ ]:
exponential_monotonocity

In [ ]:
exponential_monotonocity_inst = exponential_monotonocity.instantiate({a: two, b: t, c: Add(t, one)})

In [ ]:
# Used to allow a splitting of a summation into the sum of two summations
exponential_monotonocity_inst.derive_shifted(Neg(one))

For later summation index manipulations, we want to establish that $2^{t+1}-2^{t} = 2^{t}$ (and more specifically we will need $2^{t+1}-2^{t}-1 = 2^{t}-1$).

In [ ]:
add_one_right_in_exp

In [ ]:
two_to_quant_t_var_plus_1_factored = add_one_right_in_exp.instantiate(
        {a: two, b: t}).derive_reversed()

In [ ]:
index_shift_simplification = two_to_quant_t_var_plus_1_factored.substitution(
        subtract(Exp(two, Add(t, one)), Exp(two, t)))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().rhs.factor(
        Exp(two, t))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().rhs.simplify()

In [ ]:
index_shift_simplification = index_shift_simplification.right_add_both_sides(Neg(one))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().lhs.commute(init_idx=1, final_idx=2)

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().rhs.with_subtraction_at(1)

### Base Case

In [ ]:
base_case = induction_inst.antecedent.operands[0]

We have $|\psi_{t}\rangle$ defined as a tensor product (the result of the first phase of the quantum circuit, and the LHS of Nielsen & Chuang's identity 5.20 on pg 222):

In [ ]:
# need psi_t_var_def re-defined with a Ket instead of a NumKet on the left
from proveit.physics.quantum.QPE import _psi_t_var_as_tensor_prod
_psi_t_var_as_tensor_prod

For $\psi'_{1}$, we prove a useful equality then instantiate the `psi_t_var_as_tensor_prod` with $t=1$:

In [ ]:
psi_1_def = _psi_t_var_as_tensor_prod.instantiate({t:one})

In [ ]:
psi_1_def_simplified = psi_1_def.inner_expr().rhs.operands[1].operands[1].operands[0].exponent.simplify()

Then show that the summation formula also gives the same qbit result

In [ ]:
sum_0_to_1 = base_case.rhs

In [ ]:
sum_0_to_1_processed_01 = sum_0_to_1.inner_expr().operands[1].partitioning_first()

In [ ]:
# finish off the Base Case
base_case_jdgmt = sum_0_to_1_processed_01.sub_left_side_into(psi_1_def_simplified)

### Inductive Step

In [ ]:
inductive_step = induction_inst.antecedent.operands[1]

In [ ]:
defaults.assumptions = defaults.assumptions + inductive_step.conditions.entries

First, partition the summation:
$\sum_{k=0}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} = \sum_{k=0}^{2^{t}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} + \sum_{k=2^{t}}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1}$

In [ ]:
summation_partition_01 = (
    inductive_step.instance_expr.rhs.operands[1]
    .partitioning(two_pow_t_var_less_one))

Then shift the second summation of that partition, so that the two summations then have the same index domain:

In [ ]:
summation_partition_02 = summation_partition_01.inner_expr().rhs.operands[1].shift(Neg(Exp(two, t)))

In [ ]:
# recall the following simplification engineered earlier in the nb:
index_shift_simplification

In [ ]:
summation_partition_03 = summation_partition_02.inner_expr().rhs.operands[1].domain.upper_bound.substitute(index_shift_simplification)

We want to rewrite the summand of that 2nd summation on the rhs now by (1) expanding the exponential term and (2) rewriting the $|k+2^t{\rangle}_{t+1}$ ket as $|1\rangle \otimes |k{\rangle}_t$. This takes a little work.

In [ ]:
rhs_2nd_sum = summation_partition_03.rhs.operands[1]

In [ ]:
summand_processed_01 = rhs_2nd_sum.summand.inner_expr().operands[0].exponent.distribution(
    4, assumptions=[*defaults.assumptions,
                    InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_02 = summand_processed_01.inner_expr().rhs.operands[0].exponent_separate(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# commute the NumKet num operands to format it for replacement later using the prepend_num_ket_with_one_ket thm
summand_processed_03 = summand_processed_02.inner_expr().rhs.operands[1].num.commute(
    assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
from proveit.physics.quantum.algebra import prepend_num_ket_with_one_ket
prepend_num_ket_with_one_ket

In [ ]:
prepend_num_ket_with_one_ket_inst = prepend_num_ket_with_one_ket.instantiate(
        {n: t, k: k}, assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# can delete this cell later
summand_processed_03

In [ ]:
summand_processed_04 = (
    summand_processed_03.inner_expr().rhs.operands[1]
    .substitute(prepend_num_ket_with_one_ket_inst))

In [ ]:
# old version
# summand_processed_04 = multi_tensor_prod_induct_1_inst.sub_left_side_into(summand_processed_03)

In [ ]:
summand_processed_05 = summand_processed_04.inner_expr().rhs.operands[0].commute(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_generalized = summand_processed_05.generalize(k, domain=Interval(zero, subtract(Exp(two, t), one)))

In [ ]:
# reminder of summation_partition_03
summation_partition_03

In [ ]:
# can't seem to make this work with the old way nor the new way
# producing RuntimeError: dictionary keys changed during iteration
summation_partition_04 = (
    summation_partition_03.inner_expr().rhs.operands[1].instance_substitute(
        summand_processed_generalized))

We also then want to:

(1) pull the tensor product out of the 2nd summation, and

(2) pull the non-$k$-dependent exponential factor out of the 2nd summation.